In [1]:
!pip install unrar

In [6]:
!unrar x /content/drive/MyDrive/DoAn/dataset_B_Facial_Images.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/DoAn/dataset_B_Facial_Images.rar

Creating    dataset_B_FacialImages                                    OK
Creating    dataset_B_FacialImages/ClosedFace                         OK
Extracting  dataset_B_FacialImages/ClosedFace/closed_eye_0001.jpg_face_1.jpg       0%  OK 
Extracting  dataset_B_FacialImages/ClosedFace/closed_eye_0002.jpg_face_2.jpg       0%  OK 
Extracting  dataset_B_FacialImages/ClosedFace/closed_eye_0003.jpg_face_2.jpg       0%  OK 
Extracting  dataset_B_FacialImages/ClosedFace/closed_eye_0007.jpg_face_1.jpg       0%  OK 
Extracting  dataset_B_FacialImages/ClosedFace/closed_eye_0009.jpg_face_1.jpg       0%  OK 
Extracting  dataset_B_FacialImages/ClosedFace/closed_eye_0012.jpg_face_1.jpg       0%  OK 
Extracting  dataset_B_FacialImages/ClosedFace/closed_eye_0013.BMP_face_1.jpg       0%  OK 
Extra

In [4]:
# 1. Cài đặt thư viện dlib (có thể tốn vài phút compile)
!pip install dlib imutils opencv-python

# 2. Tải file shape_predictor_68_face_landmarks.dat (Nặng khoảng 100MB)
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

--2025-12-03 04:22:31--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-12-03 04:22:31--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  39.3MB/s    in 1.6s    

2025-12-03 04:22:33 (39.3 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [11]:
f = [f for f in os.listdir('/content/dataset_B_FacialImages/ClosedFace')]
print(len(f))
f = [f for f in os.listdir('/content/dataset_B_FacialImages/OpenFace')]
print(len(f))

1193
1232


In [13]:
import cv2
import dlib
import numpy as np
import os
from imutils import face_utils
from scipy.spatial import distance as dist
import time
# --- CẤU HÌNH ---
# Bạn cần trỏ tới folder chứa ảnh KHUÔN MẶT (Version 1 hoặc 2), không phải ảnh crop mắt
DATASET_PATH = '/content/dataset_B_FacialImages'
PREDICTOR_PATH = 'shape_predictor_68_face_landmarks.dat'
EAR_THRESHOLD = 0.25  # Ngưỡng quyết định (Bạn có thể tinh chỉnh: 0.2 - 0.3)

# --- KHỞI TẠO DLIB ---
print("⏳ Đang khởi tạo Dlib Face Detector & Predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_PATH)

# Lấy chỉ số các điểm mắt trái và phải trong bộ 68 điểm
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

def eye_aspect_ratio(eye):
    # Tính khoảng cách 2 điểm dọc (mí trên - mí dưới)
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # Tính khoảng cách ngang (khóe mắt)
    C = dist.euclidean(eye[0], eye[3])

    # Công thức EAR
    ear = (A + B) / (2.0 * C)
    return ear

def process_image(image_path):
    img = cv2.imread(image_path)
    if img is None: return None

    # Chuyển sang ảnh xám để detect nhanh hơn
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 1. Phát hiện khuôn mặt
    rects = detector(gray, 0)

    # Nếu không tìm thấy mặt nào (do ảnh mờ, tối hoặc góc nghiêng)
    if len(rects) == 0:
        return -1 # Mã lỗi: Không thấy mặt

    # Lấy khuôn mặt đầu tiên tìm thấy
    rect = rects[0]

    # 2. Tìm 68 điểm mốc
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)

    # 3. Trích xuất mắt trái và phải
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]

    # 4. Tính EAR trung bình
    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)
    avgEAR = (leftEAR + rightEAR) / 2.0

    return avgEAR

# --- BẮT ĐẦU TEST ---
classes = {'ClosedFace': 0, 'OpenFace': 1} # Lưu ý: Folder dataset của bạn phải đặt tên là closed/open
results = {
    'correct': 0,
    'total_processed': 0,
    'face_not_found': 0 # Đếm số ảnh Dlib bó tay không tìm ra mặt
}

print(f"🚀 Bắt đầu đánh giá phương pháp EAR (Threshold: {EAR_THRESHOLD})...")

for label_str, label_idx in classes.items():
    folder = os.path.join(DATASET_PATH, label_str)
    if not os.path.exists(folder):
        print(f"⚠️ Không tìm thấy folder: {folder}")
        continue

    print(f"--> Đang xử lý: {label_str}")
    for fname in os.listdir(folder):
      if fname.endswith('.jpg'):
        fpath = os.path.join(folder, fname)
        ear_value = process_image(fpath)
        if ear_value == -1:
            results['face_not_found'] += 1
            continue # Bỏ qua ảnh này

        # Rule-based Logic
        # EAR nhỏ -> Nhắm (0), EAR lớn -> Mở (1)
        pred_idx = 0 if ear_value < EAR_THRESHOLD else 1

        if pred_idx == label_idx:
            results['correct'] += 1
        results['total_processed'] += 1

# --- KẾT QUẢ ---
print("\n" + "="*40)
print(f"KẾT QUẢ PHƯƠNG PHÁP EAR (Rule-based)")
print("="*40)
if results['total_processed'] > 0:
    acc = (results['correct'] / results['total_processed']) * 100
    print(f"Số ảnh xử lý thành công: {results['total_processed']}")
    print(f"Số ảnh không tìm thấy mặt: {results['face_not_found']} (Đây là điểm yếu của Dlib)")
    print(f"Dự đoán đúng:            {results['correct']}")
    print(f"Độ chính xác (Accuracy): {acc:.2f}%")
    print(f"Độ chính xác tính tổng thể (Acc):  {(results['correct'] *100) / (results['total_processed'] + results['face_not_found'])}%" )
else:
    print("Không xử lý được ảnh nào (hoặc không tìm thấy dataset).")

⏳ Đang khởi tạo Dlib Face Detector & Predictor...
🚀 Bắt đầu đánh giá phương pháp EAR (Threshold: 0.25)...
--> Đang xử lý: ClosedFace
--> Đang xử lý: OpenFace

KẾT QUẢ PHƯƠNG PHÁP EAR (Rule-based)
Số ảnh xử lý thành công: 2217
Số ảnh không tìm thấy mặt: 206 (Đây là điểm yếu của Dlib)
Dự đoán đúng:            1901
Độ chính xác (Accuracy): 85.75%


In [15]:
print(f"Độ chính xác tính tổng thể (Acc):  {(results['correct'] *100) / (results['total_processed'] + results['face_not_found']): .2f}%" )

Độ chính xác tính tổng thể (Acc):   78.46%


In [30]:
import time
t_start = time.perf_counter() # Bấm giờ
ear_value = process_image("/content/drive/MyDrive/DoAn/test_image/opened_eye_face.jpg")
print(f"Time: {(time.perf_counter()-t_start)*1000}ms")
if ear_value == -1:
   print("Face not found")


# Rule-based Logic
# EAR nhỏ -> Nhắm (0), EAR lớn -> Mở (1)
pred_idx = 0 if ear_value < EAR_THRESHOLD else 1
print("Nhắm (0),Mở (1): ", pred_idx)


Time: 284.5483629998853ms
Nhắm (0),Mở (1):  1
